In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..'))
sys.path.append(project_root)

print(project_root)

import machine_learning.ml_lib as ML
from data_extractor.data_extractor import load_df, extract_X_y_from_df 

In [ ]:
df = load_df("../../../data/")
# Filter the data and add the column 'label' depends what you need
df = df.query("desired_answer == answer and data_type in ['REAL', 'FAKE']")
df['label'] = df.apply(lambda x: 1 if x.block_no in [1,3] else 0, axis = 1)
df.head()

In [ ]:
X,y = extract_X_y_from_df(df)

In [14]:
from datetime import datetime


param_grid = [
    {
        'classifier__penalty': ['l2'],
        'classifier__solver': ['lbfgs', 'sag'],
        'classifier__C': [0.01, 0.1, 1.0, 10.0],
        'classifier__fit_intercept': [True, False],
    },
    {
        'classifier__penalty': ['l1'],
        'classifier__solver': ['liblinear'],
        'classifier__C': [0.01, 0.1, 1.0, 10.0],
        'classifier__fit_intercept': [True, False],
    },
    {
        'classifier__penalty': ['elasticnet'],
        'classifier__solver': ['saga'],
        'classifier__C': [0.01, 0.1, 1.0],
        'classifier__l1_ratio': [0.5],
        'classifier__fit_intercept': [True],
    },
    {
        'classifier__penalty': ['none'],
        'classifier__solver': ['lbfgs', 'sag'],
        'classifier__fit_intercept': [True, False],
    }
]


result_folder = f"../results/logistic_regression_{datetime.now().strftime('%Y-%m-%d_%H:%M:%S')}/"

In [ ]:
# Invoke the function
best_model, best_params, test_score = ML.grid_search_logistic_regression_eeg(
    X,
    y,
    param_grid[0],
    test_size=0.2,
    cv=5,
    scoring='accuracy',
    random_state=42
)

In [ ]:
X.shape, y.shape

In [ ]:
from sklearn.linear_model import Ridge

param_grid_2 = {
    'regressor__alpha': [0.1, 1.0, 10.0, 100.0],  # Regularization strength
    'regressor__fit_intercept': [True, False],
    'regressor__solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg']  # Different solvers for optimization
}


In [ ]:
from sklearn.linear_model import LinearRegression

best_model_2, best_params_2, test_score_2 = ML.grid_search_linear_regression_eeg(
    X, y, param_grid_2, test_size=0.2, cv=5, scoring='neg_mean_squared_error', 
    result_folder=result_folder,
)
